In [91]:
import numpy as np
import random
import os
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.model_selection import KFold
import lightgbm as lgb
from category_encoders import TargetEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed=42
seed_everything(seed) # Seed 고정

#### 데이터 불러오기 및 전처리

In [92]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [93]:
train.head()

,ID,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
0,TRAIN_0000,P사,TayGTS,Nearly New,86.077,AWD,13642,0,No,2,159.66
1,TRAIN_0001,K사,Niro,Nearly New,56.000,FWD,10199,6,No,0,28.01
2,TRAIN_0002,A사,eT,Brand New,91.200,AWD,2361,7,No,0,66.27
3,TRAIN_0003,A사,RSeTGT,Nearly New,NaN,AWD,21683,3,No,0,99.16
4,TRAIN_0004,B사,i5,Pre-Owned,61.018,AWD,178205,1,No,0,62.02


In [94]:
# 1. 훈련 데이터에서 배터리용량 평균 계산
# '구동방식'을 그룹으로 사용하여 배터리용량 평균 계산
battery_mean_by_drive = train.groupby('구동방식')['배터리용량'].mean()
train['배터리_용량_구동방식_평균'] = train['구동방식'].map(battery_mean_by_drive)

# 2. 테스트 데이터에 매핑
# '구동방식'에 따라 배터리용량 평균값을 추가
test['배터리_용량_구동방식_평균'] = test['구동방식'].map(battery_mean_by_drive)

# 3. 훈련 데이터와 테스트 데이터 결측치 처리
# 테스트 데이터에 없는 구동방식의 결측값은 전체 평균으로 대체
global_battery_mean = train['배터리용량'].mean()
test['배터리_용량_구동방식_평균'].fillna(global_battery_mean, inplace=True)

In [95]:
train = train.drop('ID',axis=1)
test = test.drop('ID',axis=1)

### train 전처리

In [96]:
# 결측값을 각 열의 평균으로 대체
# train['배터리용량'] = train.groupby(['제조사', '모델'])['배터리용량'].transform(
#     lambda x: x.fillna(x.mean()))
# 남은 결측치는 전체 평균값으로 대체
# train['배터리용량'].fillna(train['배터리용량'].median(), inplace=True)
# train['로그_주행거리'] = np.log1p(train['주행거리(km)'])

# # 1. 훈련 데이터에 타겟 인코딩 적용
# target_encoder = TargetEncoder(cols=['배터리용량'])
# train['배터리용량_타겟'] = target_encoder.fit_transform(train['배터리용량'], train['가격(백만원)'])

### test 전처리

In [97]:
# 결측값을 각 열의 평균으로 대체
# test['배터리용량'] = test.groupby(['제조사', '모델'])['배터리용량'].transform(
#     lambda x: x.fillna(x.mean()))
# 남은 결측치는 전체 평균값으로 대체
# test['배터리용량'].fillna(test['배터리용량'].median(), inplace=True)
# test['로그_주행거리'] = np.log1p(test['주행거리(km)'])


# 2. 테스트 데이터에도 같은 매핑 값 적용
# 테스트 데이터는 따로 로드된다고 가정
# test['배터리용량_타겟'] = target_encoder.transform(test['배터리용량'])

<!-- --- -->

### category 화

In [98]:
categorical_features = [
    '제조사'
    ,'모델'
    ,'차량상태'
    ,'구동방식'
    ,'사고이력'
]
for i in categorical_features:
    train[i] = train[i].astype('category')
    test[i] = test[i].astype('category')
    
target= train['가격(백만원)']
train = train.drop('가격(백만원)', axis=1)

In [99]:
train_X, valid_X, train_y, valid_y = train_test_split(train, target, test_size=0.2, random_state=seed)

##### lgbm



In [100]:
# LightGBM 모델 정의
lgb_model = lgb.LGBMRegressor(force_row_wise=True, random_state=42)

# 랜덤 서치 하이퍼파라미터 공간 정의
param_dist = {
    'num_leaves': np.arange(5, 31, 5).tolist(),          # 리프 수 감소
    'learning_rate': np.arange(0.01, 0.5, 0.01).tolist(),# 학습 속도 줄임
    'n_estimators': np.arange(50, 301, 50).tolist(),     # 더 많은 부스팅 라운드
    'max_depth': [2, 4, 8, 16],                          # 깊이 제한 완화
    'min_data_in_leaf': np.arange(1, 11, 2).tolist(),    # 리프의 최소 데이터 수 감소
    'min_split_gain': np.arange(0.0, 0.05, 0.01).tolist(), # 최소 분할 이득
    'bagging_fraction': np.arange(0.7, 1.0, 0.1).tolist(), # 데이터 샘플링
    'feature_fraction': np.arange(0.7, 1.0, 0.1).tolist() # 피처 샘플링
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_dist,
    n_iter=500,  # 랜덤 샘플링 횟수
    scoring='neg_root_mean_squared_error',  # 평가 지표
    cv=10,  # 교차 검증 폴드 수
    verbose=1,
    random_state=42,
    n_jobs=-1  # 병렬 처리
)

# 랜덤 서치 실행
random_search.fit(train_X, train_y, categorical_feature=categorical_features)

# 최적의 하이퍼파라미터 출력
print("최적 하이퍼파라미터:", random_search.best_params_)

# 최적 모델로 테스트 데이터 예측
best_model = random_search.best_estimator_
y_pred = best_model.predict(valid_X)

# RMSE 계산
rmse = root_mean_squared_error(valid_y, y_pred)
print(f"테스트 데이터 RMSE: {rmse:.4f}")

Fitting 10 folds for each of 500 candidates, totalling 5000 fits
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.7999999999999999, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7999999999999999
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.7999999999999999, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7999999999999999
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Info] Total Bins 413
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 10
[LightGBM] [Info] Start trai

#### 예측값 출력

In [101]:
pred = best_model.predict(test)

[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.7999999999999999, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7999999999999999
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7


In [102]:
submit = pd.read_csv('sample_submission.csv')
submit['가격(백만원)'] = pred

In [103]:
import datetime
now = datetime.datetime.now()
formatted_time = now.strftime("%Y%m%d_%H%M")  
file_path = f"submission_{formatted_time}.csv"
submit.to_csv(file_path, index=False)